In [6]:
#Needed Modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

import os
import sys

import networkx as nx

from tqdm import tqdm

import json
from pathlib import Path

import matplotlib.pyplot as plt

print("Starting Notebook.")

import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm

Starting Notebook.


In [7]:
tqdm.pandas()

In [8]:
SOURCES = ['Apache', 'Hyperledger', 'IntelDAOS', 'JFrog', 'Jira', 'JiraEcosystem', 'MariaDB', 'Mindville', 'MongoDB', 'Qt', 'RedHat', 'Sakai', 'SecondLife', 'Sonatype', 'Spring']
categories = ['General Relation', 'Duplication', 'Temporal/Causal', 'Composition', 'Workflow']

In [9]:
type_dict={'Backports': 'Workflow', 
           
    'Blocked': 'Temporal/Causal',
    'Blocker': 'Temporal/Causal',
    'Blocks': 'Temporal/Causal',
           
    'Bonfire Testing': 'Workflow', 
    'Bonfire testing': 'Workflow', 
    'Git Code Review': 'Workflow', 
    'Testing': 'Workflow',
           
    'Causality': 'Temporal/Causal', 
    'Cause': 'Temporal/Causal',
    'Caused': 'Temporal/Causal', 
    'Problem/Incident': 'Temporal/Causal',
           
    'Child-Issue': 'Composition', 
    'Parent Feature': 'Composition',
    'Parent/Child': 'Composition',
    'multi-level hierarchy [GANTT]': 'Composition',
    'Parent-Relation': 'Composition',
           
    'Cloners': 'Duplication', 
    'Cloners (old)': 'Duplication', 
           
    'Collection': 'Composition', 
    'Container': 'Composition', 
    'Contains(WBSGantt)': 'Composition', 
    'Incorporate': 'Composition', 
    'Incorporates': 'Composition', 
    'Part': 'Composition',
    'PartOf': 'Composition',
    'Superset': 'Composition', 
           
    'Completes': 'Workflow', 
    'Fixes': 'Workflow',
    'Resolve': 'Workflow',
           
    'Depend': 'Temporal/Causal', 
    'Dependency': 'Temporal/Causal', 
    'Dependent': 'Temporal/Causal', 
    'Depends': 'Temporal/Causal', 
    'Gantt Dependency': 'Temporal/Causal',
    'dependent': 'Temporal/Causal',
           
    'Derived': 'Workflow',
           
    'Detail': 'Workflow', 
           
    'Documentation': 'Workflow',
    'Documented': 'Workflow',
    
    'Duplicate': 'Duplication',
           
    'Epic': 'Composition', 
    'Epic-Relation': 'Composition',
    
    'Finish-to-Finish link (WBSGantt)': 'Temporal/Causal', 
    'Gantt End to End': 'Temporal/Causal', 
    'Gantt: finish-finish': 'Temporal/Causal',
    'finish-finish [GANTT]': 'Temporal/Causal', 
    
    'Gantt End to Start': 'Temporal/Causal', 
    'Gantt: finish-start': 'Temporal/Causal',
    'finish-start [GANTT]': 'Temporal/Causal',

    'Gantt Start to Start': 'Temporal/Causal', 
    
    'Gantt: start-finish': 'Temporal/Causal', 
    
    'Follows': 'Temporal/Causal', 
    'Sequence': 'Temporal/Causal', 
    
    'Implement': 'Workflow', 
    'Implements': 'Workflow', 
    
    'Issue split': 'Composition',
    'Split': 'Composition',
    'Work Breakdown': 'Composition',
    
    'Preceded By': 'Temporal/Causal', 
    
    'Reference': 'General Relation',
    'Relate': 'General Relation',
    'Related': 'General Relation', 
    'Relates': 'General Relation',
    'Relationship': 'General Relation',
    
    'Regression': 'Workflow', 
    
    'Replacement': 'Duplication',
    
    'Required': 'Temporal/Causal', 
    
    'Supercedes': 'Workflow',
    'Supersede': 'Workflow',
    'Supersession': 'Workflow', 
    
    'Test': 'Workflow', 
    'Tested': 'Workflow',
    
    'Trigger': 'Workflow',
           
    'Subtask': 'Composition'}

In [10]:
source_lt_dict={'General Relation': ['Apache', 'Hyperledger', 'IntelDAOS', 'JFrog', 'Jira', 'JiraEcosystem', 'MariaDB', 'Mindville', 'MongoDB', 'Qt', 'RedHat', 'Sakai', 'Sonatype', 'Spring', 'SecondLife'],
    'Duplication':  ['Apache', 'Hyperledger', 'IntelDAOS', 'JFrog', 'Jira', 'JiraEcosystem', 'MariaDB', 'Mindville', 'MongoDB', 'Qt', 'RedHat', 'Sakai', 'Sonatype', 'Spring', 'SecondLife'],
    'Composition':  ['Apache', 'Hyperledger', 'IntelDAOS', 'JFrog', 'Jira', 'JiraEcosystem', 'MariaDB', 'MongoDB', 'Qt', 'RedHat', 'Sakai', 'Sonatype', 'Spring', 'SecondLife'],
    'Temporal/Causal':  ['Apache', 'Hyperledger', 'IntelDAOS', 'JFrog', 'Jira', 'JiraEcosystem', 'MariaDB', 'Mindville', 'MongoDB', 'Qt', 'RedHat', 'Sakai', 'Sonatype', 'Spring', 'SecondLife'],
    'Workflow':  ['Apache', 'Hyperledger', 'IntelDAOS', 'JFrog', 'Jira', 'JiraEcosystem', 'MongoDB', 'Qt', 'RedHat', 'Sakai', 'Sonatype', 'Spring'],
              }

In [11]:
def load_data(source):
    #Loading Issues
    filename = '../data/crawl/issues_'+source.lower()+'.csv'
    issues = pd.read_csv(filename, encoding="UTF-8", low_memory=False, sep=';')
    
    #Loading Links
    filename = '../data/crawl/clean_links_'+source.lower()+'.csv'
    links = pd.read_csv(filename, encoding="UTF-8", low_memory=False, index_col=0)
    
    return issues, links

In [12]:
link_dict = {}
issue_dict = {}

for s in SOURCES:
    print(s.upper())
    issues, links = load_data(s)
    link_dict[s] = links
    issue_dict[s] = issues
    print(f'Loaded {len(issues)} issues and {len(links)} links')

APACHE
Loaded 970929 issues and 242823 links
HYPERLEDGER
Loaded 27914 issues and 16225 links
INTELDAOS
Loaded 5557 issues and 3222 links
JFROG
Loaded 14769 issues and 3206 links
JIRA
Loaded 265343 issues and 98122 links
JIRAECOSYSTEM
Loaded 40602 issues and 10911 links
MARIADB
Loaded 31229 issues and 14618 links
MINDVILLE
Loaded 2134 issues and 44 links
MONGODB
Loaded 90629 issues and 37545 links
QT
Loaded 140237 issues and 35855 links
REDHAT
Loaded 315797 issues and 106200 links
SAKAI
Loaded 49204 issues and 19057 links
SONATYPE
Loaded 77837 issues and 4289 links
SPRING
Loaded 69100 issues and 14461 links
SECONDLIFE
Loaded 1865 issues and 630 links


In [13]:
def add_mean(df):
    
    og_df = df
    
    averages = og_df.mean()
    avg_row = averages.to_list()
    avg_row.insert(0, "Mean")
    
    stds = og_df.std()
    std_row = stds.to_list()
    std_row.insert(0, "Standard Dev.")
    
    df.loc[len(df)] = avg_row
    df.loc[len(df)] = std_row
    
    return df

In [14]:
def calc_graph_structures(issues, links):
    num_issues = len(set(issues['issue_id']))
    
    G = nx.Graph()

    print("Add Edges")
    for i in tqdm(links.index):
        G.add_edge(links['issue_id_1'].iloc[i], links['issue_id_2'].iloc[i])

    num_orphans = num_issues-G.number_of_nodes()

    components = nx.connected_components(G)
    
    assortativity = nx.degree_assortativity_coefficient(G)
    assortativity = round(assortativity, 3)

    num_comp = 0
    sum_den = 0
    sum_diam = 0
    cycle_count = 0
    star_count = 0
    tree_count = 0

    count_2_component = 0
    count_3m_component = 0
    print("Load Components")
    for c in tqdm(components):
        SG = nx.subgraph(G, c)
        n = SG.number_of_nodes()
        num_comp += 1
        if n==2:
            count_2_component += 1
        else:
            count_3m_component+=1
            diam = nx.diameter(SG)
            den = nx.density(SG)
            sum_diam += diam
            sum_den += den
            try:
                cycles = list(nx.find_cycle(SG, orientation="ignore"))
                cycle_count+=1
            except:
                tree_count+=1
                if (diam == 2) & (n>=3) & (SG.number_of_edges()==n-1):
                    star_count+=1
        
    num_issues_in_complex = round(((num_issues-num_orphans-2*count_2_component)/num_issues)*100,2)
    try:
        share_complex_components = round(((count_3m_component)/num_comp)*100,2)
    except:
        share_complex_components = 0
        
    per_orphans = round((num_orphans/num_issues)*100,2)
    
    avg_diam = sum_diam/count_3m_component
    avg_diam = round(avg_diam, 3)
    avg_dens = sum_den/count_3m_component
    avg_dens = round(avg_dens, 3)
    per_cycles = round((cycle_count/count_3m_component)*100,2)
    per_tree = round((tree_count/count_3m_component)*100,2)
    per_star = round((star_count/count_3m_component)*100,2)
    try: 
        per_2_comp = round((count_2_component/num_comp)*100,2)
        per_3_comp = round((count_3m_component/num_comp)*100,2)
    except:
        per_2_comp = 0
        per_3_comp = 0
    
    return num_issues, num_orphans, per_orphans, num_comp, count_2_component, per_2_comp, count_3m_component, per_3_comp, num_issues_in_complex, share_complex_components, assortativity, avg_diam, avg_dens, per_cycles, per_tree, per_star

In [16]:
overall_struc_df = pd.DataFrame(columns=['source', 'num_issues', 'num_orphans', 'per_orphans', 'num_comp', 'count_2_component', 'per_2_comp', 'count_3m_component', 'per_3_comp', 'num_issues_in_complex', 'share_complex_components', 'assortativity', 'avg_diam', 'avg_dens', 'per_cycles', 'per_tree', 'per_star'])
j = 0

for s in ['Hyperledger', 'IntelDAOS', 'JFrog', 'Jira', 'JiraEcosystem', 'MariaDB']:
    print(s.upper())
    issues = issue_dict[s]
    links = link_dict[s]
    num_issues, num_orphans, per_orphans, num_comp, count_2_component, per_2_comp, count_3m_component, per_3_comp, num_issues_in_complex, share_complex_components, assortativity, avg_diam, avg_dens, per_cycles, per_tree, per_star = calc_graph_structures(issues, links)

    overall_struc_df.loc[j] = [s, num_issues, num_orphans, per_orphans, num_comp, count_2_component, per_2_comp, count_3m_component, per_3_comp, num_issues_in_complex, share_complex_components, assortativity, avg_diam, avg_dens, per_cycles, per_tree, per_star]

    j+=1

 37%|███▋      | 5975/16225 [00:00<00:00, 59745.93it/s]

HYPERLEDGER
Add Edges


100%|██████████| 16225/16225 [00:00<00:00, 71807.54it/s]
0it [00:00, ?it/s]

Load Components


1381it [01:58, 11.62it/s]
100%|██████████| 3222/3222 [00:00<00:00, 76475.85it/s]
0it [00:00, ?it/s]

...

3143it [00:04, 735.61it/s] 
 49%|████▊     | 7094/14618 [00:00<00:00, 49275.37it/s]

MARIADB
Add Edges


100%|██████████| 14618/14618 [00:00<00:00, 61181.53it/s]
0it [00:00, ?it/s]

Load Components


2085it [02:25, 14.37it/s]


In [17]:
overall_struc_df

,source,num_issues,num_orphans,per_orphans,num_comp,count_2_component,per_2_comp,count_3m_component,per_3_comp,num_issues_in_complex,share_complex_components,assortativity,avg_diam,avg_dens,per_cycles,per_tree,per_star
0,Hyperledger,27914,12540,44.92,1381,653,47.28,728,52.72,50.40,52.72,-0.112,2.666,0.428,21.57,78.43,62.23
1,IntelDAOS,5557,2474,44.52,233,93,39.91,140,60.09,52.13,60.09,-0.233,2.714,0.374,24.29,75.71,62.86
2,JFrog,14769,10369,70.21,1262,835,66.16,427,33.84,18.48,33.84,-0.148,2.454,0.556,12.88,87.12,66.28
3,Jira,265342,138858,52.33,40365,31772,78.71,8593,21.29,23.72,21.29,0.003,2.689,0.534,34.82,65.18,47.62
4,JiraEcosystem,40602,27304,67.25,3143,1874,59.62,1269,40.38,23.52,40.38,-0.148,2.549,0.495,17.02,82.98,63.44
5,MariaDB,31228,17333,55.50,2085,1257,60.29,828,39.71,36.44,39.71,-0.061,2.518,0.543,24.88,75.12,57.13


In [18]:
for s in ['Mindville', 'MongoDB', 'Qt']:
    print(s.upper())
    issues = issue_dict[s]
    links = link_dict[s]
    num_issues, num_orphans, per_orphans, num_comp, count_2_component, per_2_comp, count_3m_component, per_3_comp, num_issues_in_complex, share_complex_components, assortativity, avg_diam, avg_dens, per_cycles, per_tree, per_star = calc_graph_structures(issues, links)

    overall_struc_df.loc[j] = [s, num_issues, num_orphans, per_orphans, num_comp, count_2_component, per_2_comp, count_3m_component, per_3_comp, num_issues_in_complex, share_complex_components, assortativity, avg_diam, avg_dens, per_cycles, per_tree, per_star]

    j+=1

100%|██████████| 44/44 [00:00<00:00, 40793.41it/s]
41it [00:00, 37133.76it/s]
 16%|█▌        | 6071/37545 [00:00<00:00, 60707.60it/s]

MINDVILLE
Add Edges
Load Components
MONGODB
Add Edges


100%|██████████| 37545/37545 [00:00<00:00, 67191.11it/s]
0it [00:00, ?it/s]

Load Components


7017it [12:58,  9.02it/s]
 19%|█▊        | 6679/35855 [00:00<00:00, 66778.12it/s]

QT
Add Edges


100%|██████████| 35855/35855 [00:00<00:00, 60804.93it/s]
0it [00:00, ?it/s]

Load Components


7573it [10:56, 11.53it/s] 


In [19]:
overall_struc_df

,source,num_issues,num_orphans,per_orphans,num_comp,count_2_component,per_2_comp,count_3m_component,per_3_comp,num_issues_in_complex,share_complex_components,assortativity,avg_diam,avg_dens,per_cycles,per_tree,per_star
0,Hyperledger,27914,12540,44.92,1381,653,47.28,728,52.72,50.40,52.72,-0.112,2.666,0.428,21.57,78.43,62.23
1,IntelDAOS,5557,2474,44.52,233,93,39.91,140,60.09,52.13,60.09,-0.233,2.714,0.374,24.29,75.71,62.86
2,JFrog,14769,10369,70.21,1262,835,66.16,427,33.84,18.48,33.84,-0.148,2.454,0.556,12.88,87.12,66.28
3,Jira,265342,138858,52.33,40365,31772,78.71,8593,21.29,23.72,21.29,0.003,2.689,0.534,34.82,65.18,47.62
4,JiraEcosystem,40602,27304,67.25,3143,1874,59.62,1269,40.38,23.52,40.38,-0.148,2.549,0.495,17.02,82.98,63.44
5,MariaDB,31228,17333,55.50,2085,1257,60.29,828,39.71,36.44,39.71,-0.061,2.518,0.543,24.88,75.12,57.13
6,Mindville,2134,2049,96.02,41,39,95.12,2,4.88,0.33,4.88,-0.055,2.000,0.583,0.00,100.00,100.00
7,MongoDB,90629,52025,57.40,7017,4334,61.76,2683,38.24,33.03,38.24,0.005,2.726,0.524,20.05,79.95,54.83
8,Qt,140237,99705,71.10,7573,4831,63.79,2742,36.21,22.01,36.21,-0.157,2.454,0.522,14.41,85.59,65.75


In [20]:
for s in ['RedHat', 'Sakai', 'SecondLife', 'Sonatype', 'Spring']:
    print(s.upper())
    issues = issue_dict[s]
    links = link_dict[s]
    num_issues, num_orphans, per_orphans, num_comp, count_2_component, per_2_comp, count_3m_component, per_3_comp, num_issues_in_complex, share_complex_components, assortativity, avg_diam, avg_dens, per_cycles, per_tree, per_star = calc_graph_structures(issues, links)

    overall_struc_df.loc[j] = [s, num_issues, num_orphans, per_orphans, num_comp, count_2_component, per_2_comp, count_3m_component, per_3_comp, num_issues_in_complex, share_complex_components, assortativity, avg_diam, avg_dens, per_cycles, per_tree, per_star]

    j+=1

  8%|▊         | 8040/106200 [00:00<00:01, 80388.19it/s]

REDHAT
Add Edges


100%|██████████| 106200/106200 [00:01<00:00, 70382.72it/s]
0it [00:00, ?it/s]

Load Components


28030it [30:10, 15.49it/s]  
 31%|███▏      | 5958/19057 [00:00<00:00, 59570.40it/s]

...

1723it [00:00, 6250.76it/s] 
 58%|█████▊    | 8337/14461 [00:00<00:00, 83360.74it/s]

SPRING
Add Edges


100%|██████████| 14461/14461 [00:00<00:00, 59643.41it/s]
0it [00:00, ?it/s]

Load Components


4331it [00:11, 362.54it/s] 


In [21]:
s = 'Apache'
issues = issue_dict[s]
links = link_dict[s]
num_issues, num_orphans, per_orphans, num_comp, count_2_component, per_2_comp, count_3m_component, per_3_comp, num_issues_in_complex, share_complex_components, assortativity, avg_diam, avg_dens, per_cycles, per_tree, per_star = calc_graph_structures(issues, links)

overall_struc_df.loc[j] = [s, num_issues, num_orphans, per_orphans, num_comp, count_2_component, per_2_comp, count_3m_component, per_3_comp, num_issues_in_complex, share_complex_components, assortativity, avg_diam, avg_dens, per_cycles, per_tree, per_star]

j+=1

  3%|▎         | 8092/242823 [00:00<00:02, 80911.20it/s]

Add Edges


100%|██████████| 242823/242823 [00:03<00:00, 69815.00it/s]
0it [00:00, ?it/s]

Load Components


54358it [3:05:24,  4.89it/s]  


In [22]:
add_mean(overall_struc_df)

,source,num_issues,num_orphans,per_orphans,num_comp,count_2_component,per_2_comp,count_3m_component,per_3_comp,num_issues_in_complex,share_complex_components,assortativity,avg_diam,avg_dens,per_cycles,per_tree,per_star
0,Hyperledger,27914,12540,44.920000,1381,653,47.280000,728,52.720000,50.400000,52.720000,-0.112000,2.666000,0.428000,21.570000,78.430000,62.230000
1,IntelDAOS,5557,2474,44.520000,233,93,39.910000,140,60.090000,52.130000,60.090000,-0.233000,2.714000,0.374000,24.290000,75.710000,62.860000
2,JFrog,14769,10369,70.210000,1262,835,66.160000,427,33.840000,18.480000,33.840000,-0.148000,2.454000,0.556000,12.880000,87.120000,66.280000
3,Jira,265342,138858,52.330000,40365,31772,78.710000,8593,21.290000,23.720000,21.290000,0.003000,2.689000,0.534000,34.820000,65.180000,47.620000
4,JiraEcosystem,40602,27304,67.250000,3143,1874,59.620000,1269,40.380000,23.520000,40.380000,-0.148000,2.549000,0.495000,17.020000,82.980000,63.440000
5,MariaDB,31228,17333,55.500000,2085,1257,60.290000,828,39.710000,36.440000,39.710000,-0.061000,2.518000,0.543000,24.880000,75.120000,57.130000
6,Mindville,2134,2049,96.020000,41,39,95.120000,2,4.880000,0.330000,4.880000,-0.055000,2.000000,0.583000,0.000000,100.000000,100.000000
7,MongoDB,90629,52025,57.400000,7017,4334,61.760000,2683,38.240000,33.030000,38.240000,0.005000,2.726000,0.524000,20.050000,79.950000,54.830000
8,Qt,140237,99705,71.100000,7573,4831,63.790000,2742,36.210000,22.010000,36.210000,-0.157000,2.454000,0.522000,14.410000,85.590000,65.750000
9,RedHat,315797,192905,61.090000,28030,17312,61.760000,10718,38.240000,27.950000,38.240000,-0.054000,2.650000,0.523000,21.620000,78.380000,58.040000


In [ ]:
def calc_subgraph_structures(issues, links, linktype):
    num_issues = len(set(issues['issue_id']))
           
    links_lt = links[links['mappedtype']==linktype]
    links_lt.reset_index(inplace=True)

    G = nx.Graph()

    for i in links_lt.index:
        G.add_edge(links_lt['issue_id_1'].iloc[i], links_lt['issue_id_2'].iloc[i])

    num_orphans = num_issues-G.number_of_nodes()
    try:
        assortativity = nx.degree_assortativity_coefficient(G)
        assortativity = round(assortativity, 3)
    except:
        assortativity = 0

    transitivity = nx.transitivity(G)
    transitivity = round(transitivity, 3)
    
    num_links = G.number_of_edges()

    components = nx.connected_components(G)

    num_comp = 0  
    sum_den = 0
    sum_diam = 0
    cycle_count = 0
    star_count = 0
    tree_count = 0

    count_2_component = 0
    count_3m_component = 0

    for c in components:
        SG = nx.subgraph(G, c)
        n = SG.number_of_nodes()
        num_comp += 1
        if n==2:
            count_2_component += 1
        else:
            count_3m_component+=1
            diam = nx.diameter(SG)
            den = nx.density(SG)
            sum_diam += diam
            sum_den += den
            try:
                cycles = list(nx.find_cycle(SG, orientation="ignore"))
                cycle_count+=1
            except:
                tree_count+=1
                if (diam == 2) & (n>=3) & (SG.number_of_edges()==n-1):
                    star_count+=1

#     if num_comp == 0:
#         num_comp = 1
    
#     if count_3m_component == 0:
#         count_3m_component = 1
    
    num_issues_in_complex = round(((num_issues-num_orphans-2*count_2_component)/num_issues)*100,2)
    try:
        share_complex_components = round(((count_3m_component)/num_comp)*100,2)
    except:
        share_complex_components = 0
    per_orphans = round((num_orphans/num_issues)*100,2)
    
    try:
        avg_diam = sum_diam/count_3m_component
        avg_diam = round(avg_diam, 3)
        avg_dens = sum_den/count_3m_component
        avg_dens = round(avg_dens, 3)
        per_cycles = round((cycle_count/count_3m_component)*100,2)
        per_tree = round((tree_count/count_3m_component)*100,2)
        per_star = round((star_count/count_3m_component)*100,2)
    except:
        avg_diam = 0
        avg_dens = 0
        per_cycles = 0
        per_tree = 0
        per_star = 0
    
    try:
        per_2_comp = round((count_2_component/num_comp)*100,2)
        per_3_comp = round((count_3m_component/num_comp)*100,2)
    except:
        per_2_comp = 0
        per_3_comp = 0

    return num_issues, num_orphans, per_orphans, num_comp, count_2_component, per_2_comp, count_3m_component, per_3_comp, num_issues_in_complex, share_complex_components, assortativity, transitivity, avg_diam, avg_dens, per_cycles, per_tree, per_star, num_links

In [13]:
linktype_metrics = {}

for lt in categories:
    print(lt.upper())
    lt_struc_df = pd.DataFrame(columns=['source', 'num_issues', 'num_orphans', 'per_orphans', 'num_comp', 'count_2_component', 'per_2_comp', 'count_3m_component', 'per_3_comp', 'num_issues_in_complex', 'share_complext_components', 'assortativity', 'transitivity', 'avg_diam', 'avg_dens', 'per_cycles', 'per_tree', 'per_star', 'num_edges'])
    j = 0
    source_list = source_lt_dict[lt]
    
    total_links = 0
    
    for s in source_list:
        issues = issue_dict[s]
        links = link_dict[s]
        
        links['mappedtype'] = links['linktype'].map(type_dict)
        
        num_issues, num_orphans, per_orphans, num_comp, count_2_component, per_2_comp, count_3m_component, per_3_comp, num_issues_in_complex, share_complex_components, assortativity, transitivity, avg_diam, avg_dens, per_cycles, per_tree, per_star, num_links = calc_subgraph_structures(issues, links, lt)
    
        lt_struc_df.loc[j] = [s, num_issues, num_orphans, per_orphans, num_comp, count_2_component, per_2_comp, count_3m_component, per_3_comp, num_issues_in_complex, share_complex_components, assortativity, transitivity, avg_diam, avg_dens, per_cycles, per_tree, per_star, num_links]
    
        j+=1
        
        total_links += num_links
    
    print(total_links)    
    link_dict[lt] = lt_struc_df
        

GENERAL RELATION
218833
DUPLICATION
98178
TEMPORAL/CAUSAL
84236
COMPOSITION
190913
WORKFLOW
15048


In [16]:
for lt in categories:
    add_mean(link_dict[lt])

In [27]:
def get_latex_print(df):
    
    temp = df[['source', 'per_orphans', 'per_2_comp', 'per_3_comp',
#                'transitivity', 
               'assortativity',
               'avg_dens', 'per_tree', 'per_star']]
    
    temp['per_tree'] = temp['per_tree'].apply(lambda x: str(round(x,1))+'%')
    temp['per_star'] = temp['per_star'].apply(lambda x: str(round(x,1))+'%')
    temp['per_2_comp'] = temp['per_2_comp'].apply(lambda x: str(round(x,1))+'%')
    temp['per_3_comp'] = temp['per_3_comp'].apply(lambda x: str(round(x,1))+'%')
    temp['per_orphans'] = temp['per_orphans'].apply(lambda x: str(round(x,1))+'%')
    temp['assortativity'] = temp['assortativity'].apply(lambda x: round(x,3))
#     blub['transitivity'] = blub['transitivity'].apply(lambda x: round(x,3))
    temp['avg_dens'] = temp['avg_dens'].apply(lambda x: round(x,3))
    
#     blub.at[len(blub)-2,'num_orphans'] = str(blub.loc[len(blub)-2]['num_orphans']).split(".")[0]
#     blub.at[len(blub)-2,'num_comp'] = str(blub.loc[len(blub)-2]['num_comp']).split(".")[0]
#     blub.at[len(blub)-1,'num_orphans'] = str(blub.loc[len(blub)-1]['num_orphans']).split(".")[0]
#     blub.at[len(blub)-1,'num_comp'] = str(blub.loc[len(blub)-1]['num_comp']).split(".")[0]
    print(temp.to_latex(index=False))  

In [18]:
link_dict['Composition']

,source,num_issues,num_orphans,per_orphans,num_comp,count_2_component,per_2_comp,count_3m_component,per_3_comp,num_issues_in_complex,share_complext_components,assortativity,transitivity,avg_diam,avg_dens,per_cycles,per_tree,per_star,num_edges
0,Apache,970726,852340,87.800000,16331,6683,40.920000,9648,59.080000,10.820000,59.080000,-0.120000,0.001000,2.142000,0.377000,3.370000,96.630000,90.110000,103045
1,Hyperledger,27914,15895,56.940000,1043,224,21.480000,819,78.520000,41.450000,78.520000,-0.155000,0.000000,2.215000,0.318000,0.980000,99.020000,85.100000,10994
2,IntelDAOS,5557,3034,54.600000,218,32,14.680000,186,85.320000,44.250000,85.320000,-0.316000,0.000000,2.118000,0.276000,0.000000,100.000000,90.320000,2305
3,JFrog,14769,13388,90.650000,174,106,60.920000,68,39.080000,7.920000,39.080000,-0.513000,0.000000,2.000000,0.490000,0.000000,100.000000,100.000000,1207
4,Jira,265342,258517,97.430000,1839,1132,61.560000,707,38.440000,1.720000,38.440000,-0.088000,0.002000,2.173000,0.487000,4.530000,95.470000,85.860000,5047
5,JiraEcosystem,40602,34806,85.720000,779,230,29.530000,549,70.470000,13.140000,70.470000,-0.324000,0.004000,2.213000,0.379000,4.010000,95.990000,88.340000,5093
6,MariaDB,31228,27851,89.190000,485,241,49.690000,244,50.310000,9.270000,50.310000,-0.207000,0.005000,2.340000,0.395000,7.380000,92.620000,81.150000,3009
7,MongoDB,90629,87138,96.150000,394,132,33.500000,262,66.500000,3.560000,66.500000,-0.207000,0.000000,2.034000,0.368000,0.380000,99.620000,96.560000,3098
8,Qt,140237,122316,87.220000,2498,1314,52.600000,1184,47.400000,10.910000,47.400000,-0.281000,0.001000,2.315000,0.382000,3.040000,96.960000,82.850000,15540
9,RedHat,315797,276459,87.540000,8031,3451,42.970000,4580,57.030000,10.270000,57.030000,-0.214000,0.002000,2.170000,0.447000,2.210000,97.790000,91.220000,31670


In [19]:
link_dict['General Relation']

,source,num_issues,num_orphans,per_orphans,num_comp,count_2_component,per_2_comp,count_3m_component,per_3_comp,num_issues_in_complex,share_complext_components,assortativity,transitivity,avg_diam,avg_dens,per_cycles,per_tree,per_star,num_edges
0,Apache,970726,871983,89.830000,32907,22657,68.850000,10250,31.150000,5.500000,31.150000,-0.016000,0.100000,2.592000,0.557000,16.960000,83.040000,59.790000,69284
1,Hyperledger,27914,24337,87.190000,1046,650,62.140000,396,37.860000,8.160000,37.860000,-0.051000,0.163000,2.636000,0.534000,20.960000,79.040000,60.350000,2778
2,IntelDAOS,5557,5013,90.210000,174,98,56.320000,76,43.680000,6.260000,43.680000,-0.171000,0.080000,2.368000,0.533000,18.420000,81.580000,60.530000,391
3,JFrog,14769,13343,90.340000,572,424,74.130000,148,25.870000,3.910000,25.870000,0.070000,0.111000,2.385000,0.593000,12.160000,87.840000,65.540000,873
4,Jira,265342,169401,63.840000,38639,33868,87.650000,4771,12.350000,10.630000,12.350000,0.203000,0.133000,2.711000,0.559000,37.730000,62.270000,45.550000,62749
5,JiraEcosystem,40602,36671,90.320000,1547,1142,73.820000,405,26.180000,4.060000,26.180000,0.609000,0.452000,2.291000,0.592000,14.070000,85.930000,69.380000,2540
6,MariaDB,31228,23234,74.400000,1830,1168,63.830000,662,36.170000,18.120000,36.170000,-0.040000,0.098000,2.628000,0.563000,28.100000,71.900000,51.960000,7464
7,Mindville,2134,2099,98.360000,16,14,87.500000,2,12.500000,0.330000,12.500000,-0.137000,0.000000,2.000000,0.583000,0.000000,100.000000,100.000000,19
8,MongoDB,90629,69342,76.510000,5835,3723,63.800000,2112,36.200000,15.270000,36.200000,0.088000,0.028000,2.785000,0.544000,19.030000,80.970000,55.020000,18444
9,Qt,140237,128285,91.480000,4283,3068,71.630000,1215,28.370000,4.150000,28.370000,-0.028000,0.099000,2.492000,0.566000,16.710000,83.290000,60.910000,8015


In [20]:
link_dict['Duplication']

,source,num_issues,num_orphans,per_orphans,num_comp,count_2_component,per_2_comp,count_3m_component,per_3_comp,num_issues_in_complex,share_complext_components,assortativity,transitivity,avg_diam,avg_dens,per_cycles,per_tree,per_star,num_edges
0,Apache,970726,918917,94.660000,23063,19671,85.290000,3392,14.710000,1.280000,14.710000,-0.011000,0.03200,2.178000,0.60700,4.100000,95.900000,85.050000,28910
1,Hyperledger,27914,26054,93.340000,759,590,77.730000,169,22.270000,2.440000,22.270000,-0.025000,0.01300,2.675000,0.57900,2.370000,97.630000,74.560000,1105
2,IntelDAOS,5557,5137,92.440000,160,130,81.250000,30,18.750000,2.880000,18.750000,0.306000,0.04500,3.367000,0.58300,6.670000,93.330000,73.330000,262
3,JFrog,14769,13563,91.830000,552,472,85.510000,80,14.490000,1.770000,14.490000,-0.012000,0.11200,2.013000,0.64000,6.250000,93.750000,90.000000,659
4,Jira,265342,228437,86.090000,13485,9528,70.660000,3957,29.340000,6.730000,29.340000,-0.071000,0.05800,2.244000,0.55500,9.050000,90.950000,76.950000,24060
5,JiraEcosystem,40602,37309,91.890000,1395,1128,80.860000,267,19.140000,2.550000,19.140000,-0.047000,0.02800,2.112000,0.59200,4.120000,95.880000,88.010000,1914
6,MariaDB,31228,28864,92.430000,1012,801,79.150000,211,20.850000,2.440000,20.850000,-0.043000,0.11500,2.052000,0.61600,8.060000,91.940000,83.890000,1374
7,Mindville,2134,2086,97.750000,24,24,100.000000,1,4.170000,0.000000,4.170000,NaN,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,24
8,MongoDB,90629,79478,87.700000,4575,3513,76.790000,1062,23.210000,4.550000,23.210000,-0.058000,0.02100,2.176000,0.57500,2.730000,97.270000,84.930000,6607
9,Qt,140237,129548,92.380000,4391,3341,76.090000,1050,23.910000,2.860000,23.910000,-0.058000,0.04600,2.134000,0.58900,5.240000,94.760000,84.100000,6369


In [21]:
link_dict['Temporal/Causal']

,source,num_issues,num_orphans,per_orphans,num_comp,count_2_component,per_2_comp,count_3m_component,per_3_comp,num_issues_in_complex,share_complext_components,assortativity,transitivity,avg_diam,avg_dens,per_cycles,per_tree,per_star,num_edges
0,Apache,970726,922175,95.000000,16383,11609,70.860000,4774,29.140000,2.610000,29.140000,-0.023000,0.042000,2.485000,0.537000,10.960000,89.040000,67.930000,33699
1,Hyperledger,27914,26068,93.390000,577,366,63.430000,211,36.570000,3.990000,36.570000,-0.116000,0.074000,2.502000,0.514000,18.480000,81.520000,61.140000,1341
2,IntelDAOS,5557,5197,93.520000,110,65,59.090000,45,40.910000,4.140000,40.910000,-0.183000,0.038000,2.556000,0.514000,13.330000,86.670000,57.780000,261
3,JFrog,14769,14308,96.880000,201,167,83.080000,34,16.920000,0.860000,16.920000,0.089000,0.032000,2.118000,0.572000,5.880000,94.120000,85.290000,265
4,Jira,265342,260514,98.180000,2053,1637,79.740000,416,20.260000,0.590000,20.260000,-0.034000,0.019000,2.173000,0.596000,4.810000,95.190000,83.890000,2797
5,JiraEcosystem,40602,38567,94.990000,826,625,75.670000,201,24.330000,1.930000,24.330000,0.335000,0.313000,2.259000,0.585000,9.450000,90.550000,71.140000,1265
6,MariaDB,31228,27835,89.130000,774,506,65.370000,268,34.630000,7.620000,34.630000,-0.116000,0.006000,2.601000,0.520000,10.450000,89.550000,70.150000,2771
7,Mindville,2134,2132,99.910000,1,1,100.000000,1,100.000000,0.000000,100.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1
8,MongoDB,90629,80808,89.160000,2730,1778,65.130000,952,34.870000,6.910000,34.870000,-0.089000,0.029000,2.815000,0.505000,14.500000,85.500000,60.820000,7608
9,Qt,140237,133224,95.000000,1462,929,63.540000,533,36.460000,3.680000,36.460000,-0.184000,0.007000,2.619000,0.471000,11.070000,88.930000,71.480000,5884


In [22]:
link_dict['Workflow']

,source,num_issues,num_orphans,per_orphans,num_comp,count_2_component,per_2_comp,count_3m_component,per_3_comp,num_issues_in_complex,share_complext_components,assortativity,transitivity,avg_diam,avg_dens,per_cycles,per_tree,per_star,num_edges
0,Apache,970726,956811,98.570000,6065,5070,83.590000,995,16.410000,0.390000,16.410000,-0.015000,0.005000,2.131000,0.597000,2.610000,97.39000,87.440000,7885
1,Hyperledger,27914,27901,99.950000,6,5,83.330000,1,16.670000,0.010000,16.670000,-0.167000,0.000000,2.000000,0.667000,0.000000,100.00000,100.000000,7
2,IntelDAOS,5557,5551,99.890000,3,3,100.000000,1,33.330000,0.000000,33.330000,NaN,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,3
3,JFrog,14769,14390,97.430000,177,158,89.270000,19,10.730000,0.430000,10.730000,-0.080000,0.000000,2.053000,0.618000,0.000000,100.00000,94.740000,202
4,Jira,265342,259968,97.970000,1919,1351,70.400000,568,29.600000,1.010000,29.600000,-0.114000,0.004000,2.053000,0.534000,1.230000,98.77000,94.370000,3469
5,JiraEcosystem,40602,40438,99.600000,65,47,72.310000,18,27.690000,0.170000,27.690000,-0.191000,0.000000,2.167000,0.560000,0.000000,100.00000,83.330000,99
6,MongoDB,90629,87239,96.260000,1603,1495,93.260000,108,6.740000,0.440000,6.740000,-0.012000,0.000000,2.213000,0.588000,0.930000,99.07000,81.480000,1788
7,Qt,140237,140155,99.940000,35,28,80.000000,7,20.000000,0.020000,20.000000,-0.179000,0.000000,2.000000,0.571000,0.000000,100.00000,100.000000,47
8,RedHat,315797,314938,99.730000,354,272,76.840000,82,23.160000,0.100000,23.160000,-0.091000,0.000000,2.049000,0.591000,1.220000,98.78000,93.900000,506
9,Sakai,49204,49202,100.000000,1,1,100.000000,1,100.000000,0.000000,100.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,1


In [29]:
get_latex_print(link_dict['General Relation'])

\begin{tabular}{llllrrrll}
\toprule
       source & per\_orphans & per\_2\_comp & per\_3\_comp &  transitivity &  assortativity &  avg\_dens & per\_tree & per\_star \\
\midrule
       Apache &       89.8\% &      68.8\% &      31.1\% &         0.100 &         -0.016 &     0.557 &    83.0\% &    59.8\% \\
  Hyperledger &       87.2\% &      62.1\% &      37.9\% &         0.163 &         -0.051 &     0.534 &    79.0\% &    60.4\% \\
    IntelDAOS &       90.2\% &      56.3\% &      43.7\% &         0.080 &         -0.171 &     0.533 &    81.6\% &    60.5\% \\
        JFrog &       90.3\% &      74.1\% &      25.9\% &         0.111 &          0.070 &     0.593 &    87.8\% &    65.5\% \\
         Jira &       63.8\% &      87.7\% &      12.3\% &         0.133 &          0.203 &     0.559 &    62.3\% &    45.5\% \\
JiraEcosystem &       90.3\% &      73.8\% &      26.2\% &         0.452 &          0.609 &     0.592 &    85.9\% &    69.4\% \\
      MariaDB &       74.4\% &      63.8\% &     

In [24]:
get_latex_print(link_dict['Duplication'])

\begin{tabular}{llllrrrll}
\toprule
       source & per\_orphans & per\_2\_comp & per\_3\_comp &  transitivity &  assortativity &  avg\_dens & per\_tree & per\_star \\
\midrule
       Apache &       94.7\% &      85.3\% &      14.7\% &         0.032 &         -0.011 &     0.607 &    95.9\% &    85.0\% \\
  Hyperledger &       93.3\% &      77.7\% &      22.3\% &         0.013 &         -0.025 &     0.579 &    97.6\% &    74.6\% \\
    IntelDAOS &       92.4\% &      81.2\% &      18.8\% &         0.045 &          0.306 &     0.583 &    93.3\% &    73.3\% \\
        JFrog &       91.8\% &      85.5\% &      14.5\% &         0.112 &         -0.012 &     0.640 &    93.8\% &    90.0\% \\
         Jira &       86.1\% &      70.7\% &      29.3\% &         0.058 &         -0.071 &     0.555 &    91.0\% &    77.0\% \\
JiraEcosystem &       91.9\% &      80.9\% &      19.1\% &         0.028 &         -0.047 &     0.592 &    95.9\% &    88.0\% \\
      MariaDB &       92.4\% &      79.2\% &     

In [30]:
get_latex_print(link_dict['Temporal/Causal'])

\begin{tabular}{llllrrrll}
\toprule
       source & per\_orphans & per\_2\_comp & per\_3\_comp &  transitivity &  assortativity &  avg\_dens & per\_tree & per\_star \\
\midrule
       Apache &       95.0\% &      70.9\% &      29.1\% &         0.042 &         -0.023 &     0.537 &    89.0\% &    67.9\% \\
  Hyperledger &       93.4\% &      63.4\% &      36.6\% &         0.074 &         -0.116 &     0.514 &    81.5\% &    61.1\% \\
    IntelDAOS &       93.5\% &      59.1\% &      40.9\% &         0.038 &         -0.183 &     0.514 &    86.7\% &    57.8\% \\
        JFrog &       96.9\% &      83.1\% &      16.9\% &         0.032 &          0.089 &     0.572 &    94.1\% &    85.3\% \\
         Jira &       98.2\% &      79.7\% &      20.3\% &         0.019 &         -0.034 &     0.596 &    95.2\% &    83.9\% \\
JiraEcosystem &       95.0\% &      75.7\% &      24.3\% &         0.313 &          0.335 &     0.585 &    90.5\% &    71.1\% \\
      MariaDB &       89.1\% &      65.4\% &     

In [27]:
get_latex_print(link_dict['Composition'])

\begin{tabular}{llllrrrll}
\toprule
       source & per\_orphans & per\_2\_comp & per\_3\_comp &  transitivity &  assortativity &  avg\_dens & per\_tree & per\_star \\
\midrule
       Apache &       87.8\% &      40.9\% &      59.1\% &         0.001 &         -0.120 &     0.377 &    96.6\% &    90.1\% \\
  Hyperledger &       56.9\% &      21.5\% &      78.5\% &         0.000 &         -0.155 &     0.318 &    99.0\% &    85.1\% \\
    IntelDAOS &       54.6\% &      14.7\% &      85.3\% &         0.000 &         -0.316 &     0.276 &   100.0\% &    90.3\% \\
        JFrog &       90.7\% &      60.9\% &      39.1\% &         0.000 &         -0.513 &     0.490 &   100.0\% &   100.0\% \\
         Jira &       97.4\% &      61.6\% &      38.4\% &         0.002 &         -0.088 &     0.487 &    95.5\% &    85.9\% \\
JiraEcosystem &       85.7\% &      29.5\% &      70.5\% &         0.004 &         -0.324 &     0.379 &    96.0\% &    88.3\% \\
      MariaDB &       89.2\% &      49.7\% &     

In [28]:
get_latex_print(link_dict['Workflow'])

\begin{tabular}{llllrrrll}
\toprule
       source & per\_orphans & per\_2\_comp & per\_3\_comp &  transitivity &  assortativity &  avg\_dens & per\_tree & per\_star \\
\midrule
       Apache &       98.6\% &      83.6\% &      16.4\% &         0.005 &         -0.015 &     0.597 &    97.4\% &    87.4\% \\
  Hyperledger &      100.0\% &      83.3\% &      16.7\% &         0.000 &         -0.167 &     0.667 &   100.0\% &   100.0\% \\
    IntelDAOS &       99.9\% &     100.0\% &      33.3\% &         0.000 &            NaN &     0.000 &     0.0\% &     0.0\% \\
        JFrog &       97.4\% &      89.3\% &      10.7\% &         0.000 &         -0.080 &     0.618 &   100.0\% &    94.7\% \\
         Jira &       98.0\% &      70.4\% &      29.6\% &         0.004 &         -0.114 &     0.534 &    98.8\% &    94.4\% \\
JiraEcosystem &       99.6\% &      72.3\% &      27.7\% &         0.000 &         -0.191 &     0.560 &   100.0\% &    83.3\% \\
      MongoDB &       96.3\% &      93.3\% &     

In [30]:
get_latex_print(overall_struc_df.sort_values(by='source'))

\begin{tabular}{llllrrll}
\toprule
       source & per\_orphans & per\_2\_comp & per\_3\_comp &  assortativity &  avg\_dens & per\_tree & per\_star \\
\midrule
       Apache &       71.7\% &      63.0\% &      37.0\% &         -0.061 &     0.500 &    82.5\% &    61.8\% \\
  Hyperledger &       44.9\% &      47.3\% &      52.7\% &         -0.112 &     0.428 &    78.4\% &    62.2\% \\
    IntelDAOS &       44.5\% &      39.9\% &      60.1\% &         -0.233 &     0.374 &    75.7\% &    62.9\% \\
        JFrog &       70.2\% &      66.2\% &      33.8\% &         -0.148 &     0.556 &    87.1\% &    66.3\% \\
         Jira &       52.3\% &      78.7\% &      21.3\% &          0.003 &     0.534 &    65.2\% &    47.6\% \\
JiraEcosystem &       67.2\% &      59.6\% &      40.4\% &         -0.148 &     0.495 &    83.0\% &    63.4\% \\
      MariaDB &       55.5\% &      60.3\% &      39.7\% &         -0.061 &     0.543 &    75.1\% &    57.1\% \\
         Mean &       65.1\% &      62.8\% &     